# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 2:29PM,256757,10,37547,"Digital Brands, LLC",Released
1,Feb 6 2023 2:29PM,256757,10,37549,"Digital Brands, LLC",Released
2,Feb 6 2023 2:29PM,256757,10,37548,"Digital Brands, LLC",Released
3,Feb 6 2023 2:29PM,256757,10,37550,"Digital Brands, LLC",Released
4,Feb 6 2023 2:29PM,256757,10,37551,"Digital Brands, LLC",Released
5,Feb 6 2023 2:29PM,256757,10,37552,"Digital Brands, LLC",Released
6,Feb 6 2023 2:29PM,256757,10,37553,"Digital Brands, LLC",Released
7,Feb 6 2023 2:29PM,256757,10,37554,"Digital Brands, LLC",Released
8,Feb 6 2023 2:29PM,256757,10,37555,"Digital Brands, LLC",Released
9,Feb 6 2023 2:29PM,256757,10,37556,"Digital Brands, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,256750,Released,20
31,256753,Released,1
32,256755,Released,1
33,256756,Released,1
34,256757,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256750,NaN,NaN,20.0
256753,NaN,NaN,1.0
256755,NaN,NaN,1.0
256756,NaN,NaN,1.0
256757,NaN,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0.0,0.0,2.0
256683,0.0,1.0,0.0
256684,0.0,0.0,1.0
256685,0.0,0.0,1.0
256686,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0,0,2
256683,0,1,0
256684,0,0,1
256685,0,0,1
256686,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,0,0,2
1,256683,0,1,0
2,256684,0,0,1
3,256685,0,0,1
4,256686,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,,,2
1,256683,,1,
2,256684,,,1
3,256685,,,1
4,256686,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC"
18,Feb 6 2023 2:27PM,256756,16,Sartorius Bioprocess Solutions
19,Feb 6 2023 2:17PM,256755,19,"Snap Medical Industries, LLC"
20,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation
40,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation
41,Feb 6 2023 2:08PM,256748,16,TASA USA Inc.
42,Feb 6 2023 2:01PM,256749,21,"NBTY Global, Inc."
43,Feb 6 2023 1:54PM,256746,16,Sartorius Bioprocess Solutions
44,Feb 6 2023 1:20PM,256742,15,"Alliance Pharma, Inc."
49,Feb 6 2023 1:18PM,256741,12,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",,,18
1,Feb 6 2023 2:27PM,256756,16,Sartorius Bioprocess Solutions,,,1
2,Feb 6 2023 2:17PM,256755,19,"Snap Medical Industries, LLC",,,1
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,,,20
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,,,1
5,Feb 6 2023 2:08PM,256748,16,TASA USA Inc.,,,1
6,Feb 6 2023 2:01PM,256749,21,"NBTY Global, Inc.",,,1
7,Feb 6 2023 1:54PM,256746,16,Sartorius Bioprocess Solutions,,,1
8,Feb 6 2023 1:20PM,256742,15,"Alliance Pharma, Inc.",,,5
9,Feb 6 2023 1:18PM,256741,12,"Uniderm USA, Inc",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",18,,
1,Feb 6 2023 2:27PM,256756,16,Sartorius Bioprocess Solutions,1,,
2,Feb 6 2023 2:17PM,256755,19,"Snap Medical Industries, LLC",1,,
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20,,
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,1,,
5,Feb 6 2023 2:08PM,256748,16,TASA USA Inc.,1,,
6,Feb 6 2023 2:01PM,256749,21,"NBTY Global, Inc.",1,,
7,Feb 6 2023 1:54PM,256746,16,Sartorius Bioprocess Solutions,1,,
8,Feb 6 2023 1:20PM,256742,15,"Alliance Pharma, Inc.",5,,
9,Feb 6 2023 1:18PM,256741,12,"Uniderm USA, Inc",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",18,,
1,Feb 6 2023 2:27PM,256756,16,Sartorius Bioprocess Solutions,1,,
2,Feb 6 2023 2:17PM,256755,19,"Snap Medical Industries, LLC",1,,
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20,,
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",18.0,NaN,NaN
1,Feb 6 2023 2:27PM,256756,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Feb 6 2023 2:17PM,256755,19,"Snap Medical Industries, LLC",1.0,NaN,NaN
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20.0,NaN,NaN
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",18.0,0.0,0.0
1,Feb 6 2023 2:27PM,256756,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Feb 6 2023 2:17PM,256755,19,"Snap Medical Industries, LLC",1.0,0.0,0.0
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20.0,0.0,0.0
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3080751,191.0,7.0,0.0
12,1026876,11.0,1.0,0.0
15,770172,12.0,18.0,33.0
16,1283641,4.0,1.0,0.0
19,770171,4.0,0.0,0.0
21,1026801,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3080751,191.0,7.0,0.0
1,12,1026876,11.0,1.0,0.0
2,15,770172,12.0,18.0,33.0
3,16,1283641,4.0,1.0,0.0
4,19,770171,4.0,0.0,0.0
5,21,1026801,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,191.0,7.0,0.0
1,12,11.0,1.0,0.0
2,15,12.0,18.0,33.0
3,16,4.0,1.0,0.0
4,19,4.0,0.0,0.0
5,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,191.0
1,12,Released,11.0
2,15,Released,12.0
3,16,Released,4.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,33.0,0.0,0.0,0.0
Executing,7.0,1.0,18.0,1.0,0.0,1.0
Released,191.0,11.0,12.0,4.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,33.0,0.0,0.0,0.0
1,Executing,7.0,1.0,18.0,1.0,0.0,1.0
2,Released,191.0,11.0,12.0,4.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,33.0,0.0,0.0,0.0
1,Executing,7.0,1.0,18.0,1.0,0.0,1.0
2,Released,191.0,11.0,12.0,4.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()